In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

_join = os.path.join

### Summary:

The purpose of this jupyter notebook is to extract alternate specific constants (ASC) from the tour mode choice UEC file. The ASCs vary by purpose and auto sufficiency group. 

Here are main steps:

- Read the 'ModeChoice.xlsx' for extract the ASC by mode for each purpose 
- Save the extrcated information in dataframe. 
- Convert the above ASCs into matrix files that can be added to utility OMX files in later steps. 
- Also, extract the ASCs by geogrphy - to CBD, to/from/within SF for walk to transit and drive to transit and save them in a dataframe. 
- Convert the geography ASCs to OMX files 




##### Purpose and time period definitions

In [6]:
model_year = 2050
purpose = ['Work', 'University', 'School', 'Escort', 'Shopping', 'EatOut', 
           'OthMaint', 'Social', 'OthDiscr', 'WorkBased']

time_period = {1:'EA',2:'AM',3:'MD',4:'PM',5:'EV'} #1 for EA, 2 for AM, 3 for MD, 4 for PM and 5 for EV

#### Extract ASCs by mode for each purpose 

In [3]:
#read ModeChoice.xls file
#!pip install xlrd
asc_df_final = []
const_county_final = []
for purp in purpose:
    uec_purp_columns = ['No', 'Token', 'Description', 'Filter','Formula for variable', 
                   'Index','Alt1', 'Alt2', 'Alt3', 'Alt4', 'Alt5', 'Alt6', 'Alt7', 'Alt8', 'Alt9']
    mc_purp = pd.read_excel(_join("ModeChoice.xls"), sheet_name=purp)
    mc_purp = mc_purp.iloc[2:]
    mc_purp.columns = uec_purp_columns # assign column names
    
    asc_df = mc_purp.loc[(mc_purp['Description'].str.contains('Alternative-specific constant')==True) & 
            (mc_purp['Filter'].str.contains('indivTour')==True)]
    
    asc_df = asc_df[['Description', 'Formula for variable']]
    asc_df['tour_purpose_map'] = purp
    
    asc_df[['mode', 'asc_text1', 'asc_text2', 'auto_suff']] = asc_df['Description'].str.split('-', expand=True)
    
    asc_df['auto_suff'] = asc_df['auto_suff'].str.strip()
    asc_df['mode'] = asc_df['mode'].str.strip()
    
    asc_df['auto_suff_category'] = 0
    asc_df.loc[asc_df['auto_suff']=='Zero auto', 'auto_suff_category']  = 'zeroAuto'
    asc_df.loc[asc_df['auto_suff']=='Auto deficient', 'auto_suff_category']  = 'autoDeficient'
    asc_df.loc[asc_df['auto_suff']=='Auto sufficient', 'auto_suff_category']  = 'autoSufficient'
    
    asc_df = asc_df.rename(columns={'Formula for variable': 'asc'})
    asc_df = asc_df[['mode', 'auto_suff_category', 'tour_purpose_map', 'asc']]
    asc_df_final.append(asc_df)
    
    county_const = mc_purp.loc[(mc_purp['Description'].str.contains('Walk to Transit - to CBD dummy')==True) | 
                          (mc_purp['Description'].str.contains('Drive to Transit - to CBD dummy')==True) | 
                          (mc_purp['Description'].str.contains('Walk to Transit - to San Francisco dummy')==True) | 
                          (mc_purp['Description'].str.contains('Drive to Transit - to San Francisco dummy')==True) | 
                          (mc_purp['Description'].str.contains('Walk to Transit - Within SF dummy')==True) | 
                          (mc_purp['Description'].str.contains('Drive to Transit - Within SF dummy')==True) | 
                          (mc_purp['Description'].str.contains('Walk to Transit - from San Francisco dummy')==True) | 
                          (mc_purp['Description'].str.contains('Drive to Transit - from San Francisco dummy')==True)
                          ]
    county_const = county_const[['Description', 'Formula for variable']]
    county_const.loc[county_const['Description'] == 'Walk to Transit - to CBD dummy', 'mode'] = 'wtw_cbd'
    county_const.loc[county_const['Description'] == 'Drive to Transit - to CBD dummy', 'mode'] = 'dtw_cbd'
    county_const.loc[county_const['Description'] == 'Walk to Transit - to San Francisco dummy', 'mode'] = 'wtw_to_sf'
    county_const.loc[county_const['Description'] == 'Drive to Transit - to San Francisco dummy', 'mode'] = 'dtw_to_sf'
    county_const.loc[county_const['Description'] == 'Walk to Transit - Within SF dummy', 'mode'] = 'wtw_wthn_sf'
    county_const.loc[county_const['Description'] == 'Drive to Transit - Within SF dummy', 'mode'] = 'dtw_wthn_sf'
    county_const.loc[county_const['Description'] == 'Walk to Transit - from San Francisco dummy', 'mode'] = 'wtw_from_sf'
    county_const.loc[county_const['Description'] == 'Drive to Transit - from San Francisco dummy', 'mode'] = 'dtw_from_sf'

    county_const['tour_purpose_map'] = purp
    const_county_final.append(county_const)
    
asc_df = pd.concat(asc_df_final)
county_const = pd.concat(const_county_final)

mode_names = {'Walk': 'walk_asc', 'Bike': 'bike_asc', 'Shared ride 2' : 'sr2_asc', 'Shared ride 3+' : 'sr3_asc',
              'Walk to Transit': 'wtw_asc', 'Park and ride Transit': 'PnR_asc', 'Kiss and ride Transit': 'KnR_asc',
              'Taxi': 'rh_asc'}

asc_df['mode'] = asc_df['mode'].map(mode_names)

asc_df = pd.pivot_table(asc_df, index=['auto_suff_category', 'tour_purpose_map'], columns='mode', values='asc')
asc_df = asc_df.reset_index()

county_const = pd.pivot_table(county_const, index=['tour_purpose_map'], columns='mode', values='Formula for variable')
county_const = county_const.reset_index()
#del county_const['mode']

In [4]:
asc_df

mode,auto_suff_category,tour_purpose_map,KnR_asc,PnR_asc,bike_asc,rh_asc,sr2_asc,sr3_asc,walk_asc,wtw_asc
0,autoDeficient,EatOut,-0.960,0.729,1.466,-1.962,0.606,0.624,3.415,-0.499
1,autoDeficient,Escort,-4.047,-2.570,-0.005,-0.839,1.160,1.761,0.717,-4.119
2,autoDeficient,OthDiscr,0.550,0.459,1.466,-1.226,0.606,0.312,3.205,-0.548
3,autoDeficient,OthMaint,-0.066,-0.267,0.275,-0.420,1.740,0.734,2.550,-1.230
4,autoDeficient,School,-1.081,2.407,0.714,-1.202,0.707,1.815,1.772,0.500
5,autoDeficient,Shopping,-1.228,-0.209,0.275,-0.420,1.160,1.468,3.326,-1.678
6,autoDeficient,Social,2.642,2.508,1.466,-1.226,0.404,0.624,1.872,1.817
7,autoDeficient,University,6.028,4.005,4.271,6.766,0.557,-0.209,2.172,8.742
8,autoDeficient,Work,0.280,0.260,1.481,-1.080,0.469,-0.582,3.286,-0.113
9,autoDeficient,WorkBased,-999.000,-999.000,1.637,-3.810,-2.051,-3.095,2.805,-3.808


In [5]:
county_const

mode,tour_purpose_map,dtw_cbd,dtw_from_sf,dtw_to_sf,dtw_wthn_sf,wtw_cbd,wtw_from_sf,wtw_to_sf,wtw_wthn_sf
0,EatOut,0.000,0.984,0.787,0.613,0.000,0.350,2.744,0.504
1,Escort,0.000,1.225,1.050,0.000,0.000,0.087,3.369,0.000
2,OthDiscr,0.000,0.984,0.263,0.613,0.000,0.350,1.372,0.840
3,OthMaint,0.000,0.919,0.787,0.245,0.000,0.087,2.305,0.560
4,School,0.000,0.000,0.336,0.000,0.000,0.000,1.512,0.000
5,Shopping,0.375,0.919,0.787,0.172,0.000,0.092,3.131,0.350
6,Social,0.375,1.477,0.394,0.613,0.000,0.350,2.717,0.840
7,University,1.568,1.344,2.240,0.000,1.882,0.224,5.544,0.560
8,Work,0.339,0.643,1.302,0.000,0.643,0.201,2.437,-0.300
9,WorkBased,0.000,0.000,0.000,0.000,0.282,0.188,1.410,0.000


##### convert the ASCs to OMX files

In [8]:
%%time

#constants1 folder contains ASCs with no caps. They are original constants. 

rows = 3332
columns =3332
output_dir = _join(r"V:\constants")
Path(output_dir).mkdir(parents=True, exist_ok=True)

auto_suff_cat = ['autoSufficient', 'autoDeficient', 'zeroAuto']
#auto_suff_cat = ['zeroAuto']

for auto_suff in auto_suff_cat:
    print(auto_suff)
    #purp_dict = {}
    asc_df_temp = asc_df[asc_df['auto_suff_category'] == auto_suff]
    
    for index, row in asc_df_temp.iterrows():
        tp = row['tour_purpose_map']
        KnR_asc = row['KnR_asc']
        PnR_asc = row['PnR_asc']
        bike_asc = row['bike_asc']
        rh_asc = row['rh_asc']
        sr2_asc = row['sr2_asc']
        sr3_asc = row['sr3_asc']
        walk_asc = row['walk_asc']
        WtW_asc = row['wtw_asc'] 
        
        ASC_omx = omx.open_file(_join(output_dir, f"ASC_{tp}_{auto_suff}.omx"),'w') 
        
        print(tp)
        arr_KnR_asc = np.full((rows, columns), KnR_asc)
        arr_PnR_asc = np.full((rows, columns), PnR_asc)
        arr_bike_asc = np.full((rows, columns), bike_asc)
        arr_rh_asc = np.full((rows, columns), rh_asc)
        arr_sr2_asc = np.full((rows, columns), sr2_asc)
        arr_sr3_asc = np.full((rows, columns), sr3_asc)
        arr_walk_asc = np.full((rows, columns), walk_asc)
        arr_wtw_asc = np.full((rows, columns), WtW_asc)
        
        ASC_omx['KnR_asc'] = arr_KnR_asc 
        ASC_omx['PnR_asc'] = arr_PnR_asc
        ASC_omx['rh_asc'] = arr_rh_asc
        ASC_omx['sr2_asc'] = arr_sr2_asc
        ASC_omx['sr3_asc'] = arr_sr3_asc
        ASC_omx['bike_asc'] = arr_bike_asc
        ASC_omx['walk_asc'] = arr_walk_asc
        ASC_omx['WtW_asc'] = arr_wtw_asc
        
        ASC_omx.close()

autoSufficient
EatOut
Escort
OthDiscr
OthMaint
School
Shopping
Social
University
Work
WorkBased
autoDeficient
EatOut
Escort
OthDiscr
OthMaint
School
Shopping
Social
University
Work
WorkBased
zeroAuto
EatOut
Escort
OthDiscr
OthMaint
School
Shopping
Social
University
Work
WorkBased
Wall time: 1min 7s


##### Convert geogprahy ASCs to OMX files

In [9]:
%%time
#geogrpahy constants
import pandas as pd
from itertools import product

numbers_1_to_3332 = range(1, 3333)

combinations = list(product(numbers_1_to_3332, numbers_1_to_3332))
df = pd.DataFrame(combinations, columns=['orig_taz', 'dest_taz'])

geo_cwks = pd.read_csv(_join("geographies.csv")) #columns taz, rdm_zones, super_district, county
county_dict = dict(zip(geo_cwks['taz'], geo_cwks['county']))

taz = pd.read_csv(_join('tazData_' + str(model_year) + '.csv'))
taz_atype = taz[['ZONE', 'AREATYPE']] 
atype_dict = dict(zip(taz_atype['ZONE'], taz_atype['AREATYPE']))

df['orig_county'] = df['orig_taz'].map(county_dict)
df['dest_county'] = df['dest_taz'].map(county_dict)

df['orig_atype'] = df['orig_taz'].map(atype_dict)
df['dest_atype'] = df['dest_taz'].map(atype_dict)

df['to_CBD'] = np.where(df['dest_atype'] == 1, 1,  0)
df['to_SF'] = np.where(df['dest_county'] == 1, 1,  0)
df['from_SF'] = np.where(df['orig_county'] == 1, 1,  0)
df['within_SF'] = np.where((df['orig_county'] == 1) & (df['dest_county'] == 1), 1, 0)

#create matrices 
num_zones = 3332
OD_full_index = pd.MultiIndex.from_product([range(1,num_zones + 1), range(1,num_zones + 1)])
    
    
to_CBD = df.groupby(['orig_taz', 'dest_taz'])['to_CBD'].sum().reindex(OD_full_index, fill_value=0).unstack().values
to_SF = df.groupby(['orig_taz', 'dest_taz'])['to_SF'].sum().reindex(OD_full_index, fill_value=0).unstack().values
from_SF = df.groupby(['orig_taz', 'dest_taz'])['from_SF'].sum().reindex(OD_full_index, fill_value=0).unstack().values
within_SF = df.groupby(['orig_taz', 'dest_taz'])['within_SF'].sum().reindex(OD_full_index, fill_value=0).unstack().values

Wall time: 44.1 s


In [18]:
#check 
print(np.sum(to_CBD[to_CBD > 1]))
print(np.sum(to_SF[to_SF > 1]))
print(np.sum(from_SF[from_SF > 1]))
print(np.sum(within_SF[within_SF > 1]))

0
0
0
0


In [10]:
%%time

purp_dict = {}
output_dir = _join(r"V:\constants")

for index, row in county_const.iterrows():
    tp = row['tour_purpose_map']
    dtw_cbd = row['dtw_cbd']
    dtw_from_sf = row['dtw_from_sf']
    dtw_to_sf = row['dtw_to_sf']
    dtw_wthn_sf = row['dtw_wthn_sf']
    wtw_cbd = row['wtw_cbd']
    wtw_from_sf = row['wtw_from_sf']
    wtw_to_sf = row['wtw_to_sf']
    wtw_wthn_sf = row['wtw_wthn_sf']
    
    arr_dtw_cbd = np.full((rows, columns), dtw_cbd)
    arr_dtw_from_sf = np.full((rows, columns), dtw_from_sf)
    arr_dtw_to_sf = np.full((rows, columns), dtw_to_sf)
    arr_dtw_wthn_sf = np.full((rows, columns), dtw_wthn_sf)
    arr_wtw_cbd = np.full((rows, columns), wtw_cbd)
    arr_wtw_from_sf = np.full((rows, columns), wtw_from_sf)
    arr_wtw_to_sf = np.full((rows, columns), wtw_to_sf)
    arr_wtw_wthn_sf = np.full((rows, columns), wtw_wthn_sf)
    
    walk_transit = (to_CBD * arr_wtw_cbd) + (to_SF * arr_wtw_to_sf) + (from_SF * arr_wtw_from_sf) + (within_SF * arr_wtw_wthn_sf)
    drive_transit = (to_CBD * arr_dtw_cbd) + (to_SF * arr_dtw_to_sf) + (from_SF * arr_dtw_from_sf) + (within_SF * arr_dtw_wthn_sf)
    
    geo_omx = omx.open_file(_join(output_dir, f"ASC_geography_{tp}.omx"),'w') 
    
    geo_omx['walk_transit'] = walk_transit
    geo_omx['drive_transit'] = drive_transit
    
    print(tp, np.min(walk_transit), np.max(walk_transit), np.mean(walk_transit), 
          np.min(drive_transit), np.max(drive_transit), np.mean(drive_transit))
    
    geo_omx.close()

EatOut 0.0 3.5983 0.5882968199885039 0.0 2.3844000000000003 0.34791271140809216
Escort 0.0 3.4562999999999997 0.6379425270107982 0.0 2.2750000000000004 0.4199054621848743
OthDiscr 0.0 2.5622 0.34648974800003385 0.0 1.8594000000000002 0.2510114509038901
OthMaint 0.0 2.9527 0.46070760110766884 0.0 1.9512999999999998 0.323284862474401
School 0.0 1.512 0.27907563025209836 0.0 0.336 0.06201680672268998
Shopping 0.0 3.5729 0.6067866196898876 0.0 2.2528 0.3580333022915052
Social 0.0 3.9069 0.5946861765714546 0.0 2.8579 0.40334563457735656
University 0.0 8.2096 1.2706172949852224 0.0 5.152 0.8172773109243748
Work 0.0 3.0801999999999996 0.5405808308677601 0.0 2.285 0.3928312725090029
WorkBased 0.0 1.88 0.3229627851140465 0.0 0.0 0.0
Wall time: 26.1 s


In [71]:
# End